<a href="https://colab.research.google.com/github/AlisonJD/tb_examples/blob/main/Add_a_Column_to_a_Data_Source_using_the_CLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Add a Column to a Data Source using the CLI

Based on Tinybird blog post:

https://blog.tinybird.co/2021/05/25/add-column/

In [ ]:
#@title Mount your Google Drive to save and use local files
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

% cd "/content/gdrive/My Drive/Colab Notebooks/Tinybird/tb_examples"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/Tinybird/tb_examples


In [ ]:
#@title Install Tinybird CLI, os and your token
!pip install tinybird-cli -q

import os

if not os.path.isfile('.tinyb'):
  !tb auth

if not os.path.isdir('datasources'):
  !tb init

In [ ]:
#@title Helper function to write to files
def write_text_to_file(filename, text):
  with open(filename, 'w') as f: f.write(text)

# Worked Example from Blog:

# Add a Column to a Data Source using the CLI

Business changes, so does data. New attributes in datasets are the norm, not the exception.

You can now add new columns to your existing Data Sources, without worrying about what happens with your existing data ingestion (we will keep importing data with the old schema and start accepting data with the new schema).

Since you can materialize data to other Data Sources at ingestion time, changing the schema of your Data Source could have downstream effects. Don't worry we've solved that.

## Create a Sample Data Source

In [ ]:
filename="datasources/fixtures/my_ds.csv"
text='''
n,v
1,A
2,B
3,C
4,D
5,E
6,F
7,G

'''

write_text_to_file(filename, text)

In [ ]:
!tb datasource generate datasources/fixtures/my_ds.csv --force

** Generated datasources/my_ds.datasource
** => Create it on the server running: $ tb push datasources/my_ds.datasource
** => Append data using: $ tb datasource append my_ds datasources/fixtures/my_ds.csv`

** => Generated fixture datasources/fixtures/my_ds.csv


In [ ]:
!cat datasources/my_ds.datasource

DESCRIPTION generated from datasources/fixtures/my_ds.csv

SCHEMA >
    `n` Int16,
    `v` String

In [ ]:
!tb datasource append my_ds datasources/fixtures/my_ds.csv
# the row in quaratine is the row containing column names

** 🥚 starting import process

** There was an error with file contents: 1 row in quarantine.
** 🐥 done
** Appended 8 new rows
** Total rows in my_ds: 7
** Data appended to Data Source 'my_ds' successfully!
** Data pushed to my_ds


In [ ]:
!tb sql "select * from my_ds order by n"

---------
| n | v |
---------
| 1 | A |
| 2 | B |
| 3 | C |
| 4 | D |
| 5 | E |
| 6 | F |
| 7 | G |
---------


## Add New Columns



To add new columns, just add them to the end of the current schema definition and then do `tb push --force`.

In [ ]:
filename="datasources/my_ds.datasource"
text='''
DESCRIPTION my_ds with some new columns

SCHEMA >
    `n` Int16,
    `v` String,
    `v_str_def_thing` String DEFAULT 'thing',
    `v_str_no_default` String,
    `v_int_def_3` Int16 DEFAULT 3,
    `v_int_no_default` Int16

'''

write_text_to_file(filename, text)

In [ ]:
!cat datasources/my_ds.datasource


DESCRIPTION my_ds with some new columns

SCHEMA >
    `n` Int16,
    `v` String,
    `v_str_def_thing` String DEFAULT 'thing',
    `v_str_no_default` String,
    `v_int_def_3` Int16 DEFAULT 3,
    `v_int_no_default` Int16



In [ ]:
!tb push datasources/my_ds.datasource --force

** Processing datasources/my_ds.datasource
** Building dependencies
** Running my_ds 
** The schema of 'my_ds' has changed.
**   -  ADD COLUMN `v_str_def_thing` String DEFAULT 'thing'
**   -  ADD COLUMN `v_str_no_default` String
**   -  ADD COLUMN `v_int_def_3` Int16 DEFAULT 3
**   -  ADD COLUMN `v_int_no_default` Int16
** Please confirm you want to apply the changes above y/N: y
** The Data Source has been correctly updated.
** Not pushing fixtures


## Add Data with the New Columns

In [ ]:
filename="datasources/fixtures/my_ds_new_cols.csv"
text='''
n,v,v_str_def_thing,v_str_no_default,v_int_def_3,v_int_no_default
8,H,other,word,5,10
9,I,,dog,0,6
10,J,again,words,1,8
'''

write_text_to_file(filename, text)

In [ ]:
!tb datasource append my_ds datasources/fixtures/my_ds_new_cols.csv
# again the row of column names goes into quarantine

** 🥚 starting import process

** There was an error with file contents: 1 row in quarantine.
** 🐥 done
** Appended 4 new rows
** Total rows in my_ds: 10
** Data appended to Data Source 'my_ds' successfully!
** Data pushed to my_ds


## Add Data with the Old Columns

In [ ]:
!tb sql "select * from my_ds order by n"

--------------------------------------------------------------------------------
|  n | v | v_str_def_thing | v_str_no_default | v_int_def_3 | v_int_no_default |
--------------------------------------------------------------------------------
|  1 | A | thing           |                  |           3 |                0 |
|  2 | B | thing           |                  |           3 |                0 |
|  3 | C | thing           |                  |           3 |                0 |
|  4 | D | thing           |                  |           3 |                0 |
|  5 | E | thing           |                  |           3 |                0 |
|  6 | F | thing           |                  |           3 |                0 |
|  7 | G | thing           |                  |           3 |                0 |
|  8 | H | other           | word             |           5 |               10 |
|  9 | I |                 | dog              |           0 |                6 |
| 10 | J | again           |

By default columns will have an empty string or a 0, depending on the type. In the schema you can specify other default values for the new columns.

In [ ]:
filename="datasources/fixtures/my_ds_old_cols.csv"
text='''
n,v
11,K
12,L

'''

write_text_to_file(filename, text)

In [ ]:
!tb datasource append my_ds datasources/fixtures/my_ds_old_cols.csv
# again the row of column names goes into quarantine

** 🥚 starting import process

** There was an error with file contents: 1 row in quarantine.
** 🐥 done
** Appended 4 new rows
** Total rows in my_ds: 12
** Data appended to Data Source 'my_ds' successfully!
** Data pushed to my_ds


In [ ]:
!tb sql "select * from my_ds order by n"

--------------------------------------------------------------------------------
|  n | v | v_str_def_thing | v_str_no_default | v_int_def_3 | v_int_no_default |
--------------------------------------------------------------------------------
|  1 | A | thing           |                  |           3 |                0 |
|  2 | B | thing           |                  |           3 |                0 |
|  3 | C | thing           |                  |           3 |                0 |
|  4 | D | thing           |                  |           3 |                0 |
|  5 | E | thing           |                  |           3 |                0 |
|  6 | F | thing           |                  |           3 |                0 |
|  7 | G | thing           |                  |           3 |                0 |
|  8 | H | other           | word             |           5 |               10 |
|  9 | I |                 | dog              |           0 |                6 |
| 10 | J | again           |

## Notes
1. If you are materializing views from the Data Source that you are adding columns to with `SELECT * FROM ...` the views will break because the target Data Sources won’t have all the columns. To avoid this, use column names instead of * when creating materialized views.

2. You can only add columns to Data Sources that have a `Null` engine or one in the `MergeTree` family``

3. You can keep importing data as if your schema hadn’t changed. Default values will be used for the new columns if a value is not provided for them. At any point, you can start importing with the new schema by sending data that contains the new columns.

4. All the new columns have to be added at the end of the schema of a current Data Source not in between existing columns.